<img src="Figures/top_ML.png" alt="Drawing" style="width: 1000px;"/>

# Desbalanceo de clases

<img src="Figures/scikit.jpg" alt="Drawing" style="width: 300px;"/>

https://scikit-learn.org/stable/index.html

La variable objetivo (Clase/ Etiqueta) **diabetes_binaria** tiene 2 clases:
* 0 es para no tener diabetes
* 1 es para prediabetes o diabetes.

Este conjunto de datos tiene 21 variables de características y **no está equilibrado**.

[Enlace al dataset](#https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset)

### Cargar librerías

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from sklearn.utils import resample
from sklearn.metrics import roc_auc_score,precision_recall_curve,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

import time

from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

### Cargar datos

In [ ]:
data = pd.read_csv('Data/S4_diabetes_desequilibrio_clases.csv')
data.head()

In [ ]:
data.info()

### Visualizar el desequilibrio de clases

In [ ]:
# Contamos cuantas etiquetas hay en la columna 'Diabetes'
x = data['Diabetes_binary'].value_counts()
x

In [ ]:
x = data['Diabetes_binary'].value_counts().values
x

In [ ]:
# Nombramos las etiquetas para visualizarlas
labels = ['No diabetes', 'Diabetes']
values = x

In [ ]:
# Creamos un DataFrame
df = pd.DataFrame({'Labels': labels, 'Values': values})

In [ ]:
# Creamos la gráfica
plt.figure(figsize=(8, 6))
sns.barplot(x='Labels', y='Values', data=df, palette='viridis') 
plt.title('Tabla de frecuencia de la Etiqueta')
print('Número total de etiquetas: ', x)

* Es evidente que existe un problema de desequilibrio de clases.
* Más adelante en este cuaderno estudiaremos los métodos para resolver este problema.

# Soluciones

## 1. Resampling 

Una técnica ampliamente adoptada para tratar conjuntos de datos muy desequilibrados se denomina remuestreo. Consiste en eliminar muestras de la clase mayoritaria (submuestreo) y/o añadir más ejemplos de la clase minoritaria (sobremuestreo).

<img src="Figures/resampling.png" alt="Drawing" style="width: 800px;"/>


### **1.1 Oversampling de la clase minoritaria**

El sobremuestreo puede definirse como añadir más copias de la clase minoritaria. El sobremuestreo puede ser una buena opción cuando no se tiene muchos datos con los que trabajar.

Utilizaremos el módulo de remuestreo de Scikit-Learn para replicar aleatoriamente muestras de la clase minoritaria.

In [ ]:

no_diabetes = data[data.Diabetes_binary == 0]
diabetes = data[data.Diabetes_binary == 1]

# upsample minority
diabetes_upsampled = resample(diabetes,
                          replace=True, # sample with replacement
                          n_samples=len(no_diabetes), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([no_diabetes, diabetes_upsampled])

In [ ]:
upsampled

In [ ]:
x = upsampled['Diabetes_binary'].value_counts().values
x

In [ ]:
# Creamos un DataFrame
df_upsampled = pd.DataFrame({'Labels': labels, 'Values': x})

# Creamos la gráfica
plt.figure(figsize=(8, 6))
sns.barplot(x='Labels', y='Values', data=df_upsampled, palette='viridis') 
plt.title('Tabla de frecuencia de la Etiqueta UPSAMPLED')
print('Número total de etiquetas: ', x)

In [ ]:
# check the dataset
upsampled

### **1.2 Undersampling la clase mayoritaria**

El **submuestreo** puede definirse como la **eliminación de algunas observaciones de la clase mayoritaria**. El submuestreo puede ser una buena opción cuando se dispone de una gran cantidad de datos, por ejemplo, millones de filas. Pero el inconveniente es que estamos eliminando información que puede ser valiosa. Esto podría conducir a un ajuste insuficiente y a una mala generalización en el conjunto de pruebas.


In [ ]:
no_diabetes_downsampled = resample(no_diabetes,
                                replace = False, # sample without replacement
                                n_samples = len(diabetes), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([no_diabetes_downsampled, diabetes])

# checking counts
downsampled.Diabetes_binary.value_counts()

In [ ]:
x = downsampled['Diabetes_binary'].value_counts().values
x

In [ ]:

# Creamos un DataFrame
df_downsampled = pd.DataFrame({'Labels': labels, 'Values': x})

# Creamos la gráfica
plt.figure(figsize=(8, 6))
sns.barplot(x='Labels', y='Values', data=df_downsampled, palette='viridis') 
plt.title('Tabla de frecuencia de la Etiqueta DOWNSAMPLED')
print('Número total de etiquetas: ', x)

In [ ]:
# check the dataset
downsampled

## 2. Datos sintéticos

En la literatura se han propuesto varias técnicas de remuestreo más sofisticadas.

Por ejemplo, en el sobremuestreo, en lugar de crear copias exactas de los registros de la clase minoritaria, podemos introducir pequeñas variaciones en esas copias, creando muestras sintéticas más diversas.

Vamos a aplicar esta técnica de remuestreo (creación de datos sintéticos), utilizando la biblioteca de Python **[imbalanced-learn](https://imbalanced-learn.org/stable/)**. Es compatible con scikit-learn y forma parte de los proyectos scikit-learn-contrib.

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
import imblearn

### Over-sampling: SMOTE

SMOTE (Synthetic Minority Oversampling TEchnique) consiste en sintetizar elementos para la clase minoritaria, basándose en los que ya existen. Funciona eligiendo aleatoriamente un punto de la clase minoritaria y calculando los k vecinos más cercanos para este punto. Los puntos sintéticos se añaden entre el punto elegido y sus vecinos.

<img src="Figures/SMOTE.png" alt="Drawing" style="width: 800px"/>


## 3. Cambiar la métrica

Como hemos visto anteriormente, el acierto (Accuracy) no es la mejor métrica para evaluar conjuntos de datos desequilibrados, ya que puede ser engañosa. Entre las métricas que pueden proporcionar una mejor visión se encuentran:

* **Matriz de confusión**: una tabla que muestra las predicciones correctas y los tipos de predicciones incorrectas.

* **Precisión**: el número de verdaderos positivos dividido por todas las predicciones positivas. La precisión también se denomina valor predictivo positivo. Es una medida de la exactitud de un clasificador. Una precisión baja indica un elevado número de falsos positivos.

* **Recall**: el número de verdaderos positivos dividido por el número de valores positivos en los datos de prueba.También se denomina sensibilidad o tasa de verdaderos positivos. Es una medida de la exhaustividad de un clasificador. Un recall bajo indica un alto número de falsos negativos.

* **Puntuación F1**: la media ponderada de la precisión y la recuperación.
